In [4]:
import os
import pandas as pd
import numpy as np
import seaborn as sns
import cv2
from glob import glob
import tensorflow as tf
import matplotlib.pyplot as plt
from PIL import Image
from tensorflow.keras.preprocessing import image
from matplotlib.image import imread
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.layers import BatchNormalization
from sklearn.metrics import classification_report,confusion_matrix
from tensorflow.keras.models import Sequential, Model
from keras.regularizers import l2
from tensorflow.keras.layers import Activation, Dropout, Dense, Flatten, Conv2D, BatchNormalization, MaxPooling2D, GlobalAveragePooling2D,Input,concatenate, Lambda
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.applications.inception_v3 import preprocess_input
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

import warnings
warnings.filterwarnings("ignore")

In [5]:
# Main Folder Path
folder_path = "/kaggle/input/kermany2018/OCT2017 "

# Sub Folder Paths
train_dir = f"{folder_path}/train"
val_dir = f"{folder_path}/val"
test_dir = f"{folder_path}/test"

In [6]:
os.listdir(folder_path)

['val', 'test', 'train']

In [7]:
print(f"Train Directory: {os.listdir(train_dir)}")
print(f"Validation Directory: {os.listdir(test_dir)}")
print(f"Test Directory: {os.listdir(val_dir)}")

Train Directory: ['DRUSEN', 'CNV', 'NORMAL', 'DME']
Validation Directory: ['DRUSEN', 'CNV', 'NORMAL', 'DME']
Test Directory: ['DRUSEN', 'CNV', 'NORMAL', 'DME']


In [8]:
# === Column 2: Data generators (IRv2) + class weights ===
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np, os

IMG_SIZE   = (299, 299)   # Inception-ResNet-v2 native
BATCH_SIZE = 32

train_aug = ImageDataGenerator(
    rotation_range=10,
    width_shift_range=0.05,
    height_shift_range=0.05,
    zoom_range=0.10,
    horizontal_flip=True,
)
eval_aug  = ImageDataGenerator()

train_gen = train_aug.flow_from_directory(
    train_dir, target_size=IMG_SIZE, color_mode="rgb",
    batch_size=BATCH_SIZE, class_mode="categorical", shuffle=True
)
val_gen = eval_aug.flow_from_directory(
    val_dir, target_size=IMG_SIZE, color_mode="rgb",
    batch_size=BATCH_SIZE, class_mode="categorical", shuffle=False
)
test_gen = eval_aug.flow_from_directory(
    test_dir, target_size=IMG_SIZE, color_mode="rgb",
    batch_size=BATCH_SIZE, class_mode="categorical", shuffle=False
)

# Compute class weights: total / (n_classes * count_c)
counts = np.zeros(train_gen.num_classes, dtype=np.int64)
for cls, idx in train_gen.class_indices.items():
    counts[idx] = len(os.listdir(os.path.join(train_dir, cls)))
total = int(counts.sum()); ncls = len(counts)
class_weights = {i: float(total/(ncls*counts[i])) for i in range(ncls)}

print("Counts:", counts.tolist())
print("Class weights:", class_weights)


Found 83484 images belonging to 4 classes.
Found 32 images belonging to 4 classes.
Found 968 images belonging to 4 classes.
Counts: [37205, 11348, 8616, 26315]
Class weights: {0: 0.5609729875016799, 1: 1.8391787099048291, 2: 2.4223537604456826, 3: 0.7931217936538096}


In [9]:
# === Column 3: Inception-ResNet-v2 model + Stage-1 (frozen) ===
import tensorflow as tf
from tensorflow.keras import layers, Model, optimizers
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, CSVLogger

num_classes = train_gen.num_classes

# Input + in-model rescale to [-1, 1] (keeps generators vanilla)
inp = layers.Input(shape=(*IMG_SIZE, 3), name="image")
x   = layers.Rescaling(1./127.5, offset=-1)(inp)

# Robust weights load (falls back to random init if download blocked)
try:
    base = tf.keras.applications.InceptionResNetV2(
        include_top=False, weights="imagenet", input_tensor=x
    )
    print("IRv2 ImageNet weights loaded.")
except Exception as e:
    print("IRv2 weights download issue -> random init used.", e)
    base = tf.keras.applications.InceptionResNetV2(
        include_top=False, weights=None, input_tensor=x
    )

y   = layers.GlobalAveragePooling2D()(base.output)
y   = layers.Dropout(0.3)(y)
out = layers.Dense(num_classes, activation="softmax")(y)
model = Model(inp, out, name="IRv2_OCT")
model.summary()

# Stage-1: freeze backbone
base.trainable = False
loss = tf.keras.losses.CategoricalCrossentropy(label_smoothing=0.10)

model.compile(optimizer=optimizers.Adam(1e-3),
              loss=loss, metrics=["accuracy"])

ckpt_s1 = "IRv2_stage1_best.keras"
cb_s1 = [
    EarlyStopping(monitor="val_loss", patience=4, restore_best_weights=True, verbose=1),
    ReduceLROnPlateau(monitor="val_loss", factor=0.3, patience=2, verbose=1),
    ModelCheckpoint(ckpt_s1, monitor="val_accuracy", save_best_only=True, verbose=1),
    CSVLogger("IRv2_stage1_log.csv", append=False),
]

EPOCHS_S1 = 8
history_s1 = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=EPOCHS_S1,
    class_weight=class_weights,
    callbacks=cb_s1,
    verbose=1,
)


I0000 00:00:1763330364.144633      48 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15513 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0


219055592/219055592 ━━━━━━━━━━━━━━━━━━━━ 6s 0us/step
IRv2 ImageNet weights loaded.


Model: "IRv2_OCT"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ image (InputLayer)  │ (None, 299, 299,  │          0 │ -                 │
│                     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ rescaling           │ (None, 299, 299,  │          0 │ image[0][0]       │
│ (Rescaling)         │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 149, 149,  │        864 │ rescaling[0][0]   │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalization │ (None, 149, 149,  │         96 │ conv2d[0][0]      │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation          │ (None, 149, 149,  │          0 │ batch_normalizat… │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1 (Conv2D)   │ (None, 147, 147,  │      9,216 │ activation[0][0]  │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 147, 147,  │         96 │ conv2d_1[0][0]    │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_1        │ (None, 147, 147,  │          0 │ batch_normalizat… │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_2 (Conv2D)   │ (None, 147, 147,  │     18,432 │ activation_1[0][… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 147, 147,  │        192 │ conv2d_2[0][0]    │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_2        │ (None, 147, 147,  │          0 │ batch_normalizat… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d       │ (None, 73, 73,    │          0 │ activation_2[0][… │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_3 (Conv2D)   │ (None, 73, 73,    │      5,120 │ max_pooling2d[0]… │
│                     │ 80)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 73, 73,    │        240 │ conv2d_3[0][0]    │
│ (BatchNormalizatio… │ 80)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_3        │ (None, 73, 73,    │          0 │ batch_normalizat… │
│ (Activation)        │ 80)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_4 (Conv2D)   │ (None, 71, 71,    │    138,240 │ activation_3[0][… │
│                     │ 192)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 71, 71,    │        576 │ conv2d_4[0][0]  

 Total params: 54,342,884 (207.30 MB)

...

Epoch 1/8


I0000 00:00:1763330393.924921     134 service.cc:148] XLA service 0x7948e0002720 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1763330393.925921     134 service.cc:156]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0
I0000 00:00:1763330398.358837     134 cuda_dnn.cc:529] Loaded cuDNN version 90300


   1/2609 ━━━━━━━━━━━━━━━━━━━━ 21:50:47 30s/step - accuracy: 0.0625 - loss: 1.8128

I0000 00:00:1763330407.698674     134 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


2609/2609 ━━━━━━━━━━━━━━━━━━━━ 0s 857ms/step - accuracy: 0.6323 - loss: 1.0532
Epoch 1: val_accuracy improved from -inf to 0.90625, saving model to IRv2_stage1_best.keras
2609/2609 ━━━━━━━━━━━━━━━━━━━━ 2280s 863ms/step - accuracy: 0.6323 - loss: 1.0532 - val_accuracy: 0.9062 - val_loss: 0.6239 - learning_rate: 0.0010
Epoch 2/8
2609/2609 ━━━━━━━━━━━━━━━━━━━━ 0s 669ms/step - accuracy: 0.7203 - loss: 0.9183
Epoch 2: val_accuracy did not improve from 0.90625
2609/2609 ━━━━━━━━━━━━━━━━━━━━ 1747s 670ms/step - accuracy: 0.7203 - loss: 0.9183 - val_accuracy: 0.8750 - val_loss: 0.6036 - learning_rate: 0.0010
Epoch 3/8
2609/2609 ━━━━━━━━━━━━━━━━━━━━ 0s 663ms/step - accuracy: 0.7315 - loss: 0.9091
Epoch 3: val_accuracy did not improve from 0.90625
2609/2609 ━━━━━━━━━━━━━━━━━━━━ 1731s 664ms/step - accuracy: 0.7315 - loss: 0.9091 - val_accuracy: 0.8750 - val_loss: 0.6151 - learning_rate: 0.0010
Epoch 4/8
2609/2609 ━━━━━━━━━━━━━━━━━━━━ 0s 664ms/step - accuracy: 0.7269 - loss: 0.9137
Epoch 4: ReduceL